In [1]:
import numpy as np
import pandas as pd
import util

In [2]:
import yaml
#API KEY
with open("../security.yaml") as f:
    SYS_CONFIG = yaml.safe_load(f)

# `SubwayBiStation-IntervalTime.ipynb`
> API 정보 - https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15001019#tab_layer_detail_function\
> `소모시간` = 도시철도가 어떤 역에서 다른 역까지 이동하는데 걸리는 순수 시간\
> `정차시간` = 도시철도가 어떤 역에서 도착한 후 정지하여 다음 역으로 출발하기 전 까지의 순수 시간\
> `실질배차간격` = `철도사용자의최악대기시간(환승시간제외)` = 철도사용자가 한 역에서 어떤 열차가 지나가는 것을 보고 다음 열차가 올 때까지의 시간
- 두 역에서의 `소모시간`을 데이터
- 한 역에서의 `정차시간`을 데이터

## 1. `소모시간`, `정차시간` 을 알기 위해 API로 데이터 Loading

```py
ENCODING        "EUC-KR"
startSn         출발역명:str
startSc	        출발역코드:int      (ex.신평:101, 안평:414)
endSn	        도착역명:str
endSc	        도착역코드:int      (ex.신평:101, 안평:414)
dist	        이동거리:int        단위:0.1 km (ex 16->1.6 km)
time	        이동시간:int        단위:초(sec)
stoppingTime	정차시간:int        단위:초(sec)
exchange        환승구분:str|Null   "Y" => 환승역
                                   "N" => 경전철
                                   ""(공란) => 일반역```


In [3]:
# GET API REQ
url = "http://data.humetro.busan.kr/voc/api/open_api_distance.tnn"
params = {"serviceKey": SYS_CONFIG["API_KEY"], "act": "json", "numOfRows": 328}

import requests

response = requests.get(url, params=params)

In [4]:
# TRIM AND SAVE REQ
import json


# SET JSON DECODER
# https://stackoverflow.com/questions/45068797/how-to-convert-string-int-json-into-real-int-with-json-loads
class Decoder(json.JSONDecoder):
    def decode(self, s):
        result = super().decode(
            s
        )  # result = super(Decoder, self).decode(s) for Python 2.x
        return self._decode(result)

    def _decode(self, o):
        if isinstance(o, str):
            if o == "Y":
                return True
            elif o == "N":
                return False
            elif o == "":
                return None

            try:
                return int(o)
            except ValueError:
                return o
        elif isinstance(o, dict):
            return {k: self._decode(v) for k, v in o.items()}
        elif isinstance(o, list):
            return [self._decode(v) for v in o]
        else:
            return o

In [5]:
# TRIM AND SAVE REQ
df_interstation_info = json.loads(
    req := response.content.decode("EUC-KR"), cls=Decoder
)
df_interstation_info = df_interstation_info["response"]["body"]["item"]
df_interstation_info = pd.DataFrame(df_interstation_info)
df_interstation_info = df_interstation_info.loc[:,["dist","startSn","endSn","exchange","startSc","endSc","stoppingTime","time"]] #순서변경
df_interstation_info = df_interstation_info[(df_interstation_info["startSc"] < 500) & (df_interstation_info["endSc"] < 500)]

PATH_INTER_STATION_SPEND_TIME = "../trimmed_data/IntervalTimeAPI.pickle"
util.savePickle(df_interstation_info,PATH_INTER_STATION_SPEND_TIME)
interval_time = util.loadPickle(PATH_INTER_STATION_SPEND_TIME)

PATH_INTER_STATION_EXCHANGE_TIME = "../trimmed_data/ExchangeTimeAPI.pickle"
df_exchange_time = interval_time[interval_time["exchange"]==True]
util.savePickle(df_exchange_time,PATH_INTER_STATION_EXCHANGE_TIME)
exchange_time = util.loadPickle(PATH_INTER_STATION_EXCHANGE_TIME)

[interval_time.head(3),exchange_time.head(3)]

[   dist startSn endSn  exchange  startSc  endSc  stoppingTime  time
 0    17      동매    신평     False      100    101            30   180
 1    12      동매    장림     False      100     99            20   180
 2    17      신평    동매     False      101    100            30   180,
     dist startSn endSn  exchange  startSc  endSc  stoppingTime  time
 40     1      서면    서면      True      119    219             0   120
 50     1      연산    연산      True      123    305             0   180
 56     3      동래    동래      True      125    402             0   360]

## 2. 역 to 역 시간 구하기
- 시작역, 끝역
- 무정차대상역들
  - IF 지정X THEN 일반노선에서 소모 시간 RETURN
```
attributes = [
    "startSn",
    "startSc",
    "endSn",
    "endSc",
    "dist",
    "time",
    "stoppingTime",
    "exchange",
]
```

In [6]:
def data_interstation_spend_time():
    import util
    interval_time = util.loadPickle(util.PATH_INTER_STATION_SPEND_TIME)

    subway_graph = interval_time.loc[
        :, ["exchange", "startSc", "endSc", "stoppingTime", "time"]
    ]
    subway_graph = subway_graph.groupby(by=["startSc", "endSc", "exchange"]).sum()
    print("len(subway_graph) == len(interval_time) : ",len(subway_graph) == len(interval_time))

    subway_graph = subway_graph.T.to_dict()
    subway_graph_index = list(subway_graph.keys())
    return subway_graph_index, subway_graph

def line_range(sCode_x):
    if sCode_x < 200:
        return range(0, 200)
    elif sCode_x < 300:
        return range(200, 300)
    elif sCode_x < 400:
        return range(300, 400)
    elif sCode_x < 500:
        return range(400, 500)
    else:
        assert False, "MUST sCode_x < 500 (4호선까지 존재 합니다.)"


def isExchange(sCode_x, sCode_y):
    """경로에서 환승이 일어 났는지 확인하는 함수"""
    assert sCode_x > 0 and sCode_y > 0, ValueError
    _X, _Y = sCode_x // 100, sCode_y // 100
    if _X > 1 and _Y > 1:
        return _X != _Y
    elif _X <= 1 and _Y <= 1:
        return False
    else:
        return True
    
isExchange(97,312)

True

In [7]:
subway_graph_index, subway_graph = data_interstation_spend_time()

# 환승정보역 관계를 갖는 index 추출
exchange_relation_index = []
for _index in subway_graph_index:
    _, _, exchange = _index
    if exchange is True:
        exchange_relation_index.append(_index)
print(*exchange_relation_index)
print(subway_graph)


# exchange_relation = []
# for startSc,endSc,exchange in subway_graph_index:
#     _index = (startSc,endSc,exchange)
#     if exchange is True:
#         exchange_relation.append(_index)
# exchange_relation

sCode_x, sCode_y = 97, 312


def find_exchange_inline(sCode: int):
    result_arr = []
    for _index in exchange_relation_index:
        startSc, _, _ = _index
        if startSc in line_range(sCode):
            result_arr.append(startSc)
    return result_arr


def find_same_station(sCode: int):  # -> list[Any]:
    _same_stations = []
    for _index in exchange_relation_index:
        startSc, endSc, _ = _index
        if sCode == startSc:
            _same_stations.append(endSc)
    return _same_stations


def find_exchange_near(sCode: int):
    _exchange_candidate = find_exchange_inline(sCode)
    _diff = [abs(s - sCode) for s in _exchange_candidate]
    _result_index = _diff.index(min(_diff))
    _near_station = [_exchange_candidate[_result_index]]
    _near_station.extend(find_same_station(_near_station[0]))
    return _near_station


def make_RL_Node_inline(sCode: int):
    l_node, r_node = [], []
    for s in find_exchange_inline(sCode):
        print(s,sCode)
        if sCode < s:
            r_node.append(s)
            print(s,"s at R")
        elif s < sCode:
            l_node.append(s)
    return l_node, r_node
        
make_RL_Node_inline(309)

# print(
#     find_exchange_inline(sCode_x),
#     find_same_station(sCode_x),
#     find_exchange_near(sCode_x),
#     find_exchange_inline(sCode_y),
#     find_same_station(sCode_y),
#     find_exchange_near(sCode_y),
# )

len(subway_graph) == len(interval_time) :  True
(119, 219, True) (123, 305, True) (125, 402, True) (208, 301, True) (219, 119, True) (233, 313, True) (301, 208, True) (305, 123, True) (309, 401, True) (313, 233, True) (401, 309, True) (402, 125, True)
{(95, 96, False): {'stoppingTime': 20, 'time': 180}, (96, 95, False): {'stoppingTime': 20, 'time': 180}, (96, 97, False): {'stoppingTime': 20, 'time': 120}, (97, 96, False): {'stoppingTime': 20, 'time': 120}, (97, 98, False): {'stoppingTime': 20, 'time': 120}, (98, 97, False): {'stoppingTime': 20, 'time': 120}, (98, 99, False): {'stoppingTime': 20, 'time': 120}, (99, 98, False): {'stoppingTime': 20, 'time': 120}, (99, 100, False): {'stoppingTime': 20, 'time': 180}, (100, 99, False): {'stoppingTime': 20, 'time': 180}, (100, 101, False): {'stoppingTime': 30, 'time': 180}, (101, 100, False): {'stoppingTime': 30, 'time': 180}, (101, 102, False): {'stoppingTime': 0, 'time': 140}, (102, 101, False): {'stoppingTime': 30, 'time': 150}, (102, 103,

([301, 305], [313])